# IV approch for Employee presence

This document presents several approaches to try to use an IV approach for employee presence and its interaction with treatment. Unfortunately, the time of the hearing is a very weak instrument for employee presence (F-stat $< 4$) and the coefficients of the main table are not signifficant. 

In [ ]:
clear
set more off
qui cd "D:\MCLC\Pilot 1\information_sett\ReplicationFinal"
//qui do ".\DoFiles\tabnumlab.do"

### Limpieza de la base

In [ ]:
// Loading and cleaning the database

local controls i.anio i.junta i.phase i.numActores
local balance_var gen trabajador_base horas_sem c_antiguedad abogado_pub reinst indem salario_diario sal_caidos prima_antig hextra rec20  prima_dom  desc_sem desc_ob sarimssinf utilidades nulidad min_ley 

qui use "./DB/scaleup_operation.dta", clear
rename año anio
rename expediente exp
qui merge m:1 junta exp anio using "./DB/scaleup_casefiles_wod.dta" , nogen  keep(1 3)

*Notified casefiles
qui keep if notificado==1

*Homologation
qui gen treatment=.
qui replace treatment=2 if dia_tratamiento==1
qui replace treatment=1 if dia_tratamiento==0
rename convenio seconcilio
rename convenio_seg_2m convenio_2m 
rename convenio_seg_5m convenio_5m
qui gen fecha=date(fecha_lista,"YMD")
format fecha %td


*Time hearing (Instrument)
qui gen time_hearing=substr(horario_aud,strpos(horario_aud," "),length(horario_aud))
qui egen time_hr=group(time_hearing)

keep seconcilio convenio_2m convenio_5m fecha junta exp anio fecha treatment p_actor abogado_pub time_hearing time_hr numActores `balance_var'
qui gen phase=2
tempfile p2
qui save `p2'

use "./DB/pilot_operation.dta" , clear	
qui replace junta=7 if missing(junta)

*Presence employee
qui replace p_actor=(p_actor==1)
*Not in experiment
qui drop if tratamientoquelestoco==0
rename tratamientoquelestoco treatment
rename expediente exp

*Time hearing (Instrument)
qui gen time_hearing=substr(horarioaudiencia,strpos(horarioaudiencia," "),length(horarioaudiencia))
qui egen time_hr=group(time_hearing)

qui keep seconcilio convenio_2m convenio_5m fecha junta exp anio fecha treatment p_actor abogado_pub time_hearing time_hr numActores `balance_var'
qui append using `p2'
qui replace phase=1 if missing(phase)

qui merge m:1 junta exp anio using ".\DB\seguimiento_m5m.dta", nogen
qui merge m:1 junta exp anio using ".\Terminaciones\Data\followUps2020.dta", gen(merchados) keep(1 3)
qui replace seconcilio = 0 if modoTermino != 3 & !missing(modoTermino)
********************************************************************************

bysort junta exp anio: gen DuplicatesPredrop=_N
forvalues i=1/3{
	qui gen T`i'_aux=[treatment==`i'] 
	qui bysort junta exp anio: egen T`i'=max(T`i'_aux)
}

qui gen T1T2=[T1==1 & T2==1]
qui gen T1T3=[T1==1 & T3==1]
qui gen T2T3=[T2==1 & T3==1]
qui gen TAll=[T1==1 & T2==1 & T3==1]

qui replace T1T2=0 if TAll==1
qui replace T1T3=0 if TAll==1
qui replace T2T3=0 if TAll==1
*32 drops
*drop if T1T2==1 & treatment == 1
*46 drops
qui drop if T1T3==1
*31 drops
qui drop if T2T3==1
*8 drops
qui drop if TAll==1

*bysort junta exp anio: gen DuplicatesPostdrop=_N

*44 drops
*sort junta exp anio fecha
*bysort junta exp anio: keep if _n==1
********************************************************************************

sort junta exp anio fecha
qui by junta exp anio: gen renglon = _n
qui keep if renglon==1

*Drop conciliator observations
qui drop if treatment==3

qui replace anio = 2010 if anio < 2010
qui replace numActores = 3 if numActores>3

global controls i.anio i.junta i.phase i.numActores
qui gen treatment2  = treatment -1


## Instrumented by "extreme"  hours: before 10 am or between 12 and 1 pm

In [ ]:
gen time_instrument=inlist(time_hr,1,2,7,8) if !missing(time_hr) 
ivregress 2sls seconcilio treatment2 $controls (1.p_actor 1.p_actor#1.treatment2 = 1.time_instrument 1.time_instrument#1.treatment2), first

## Instrumented by "super extreme"  hours: before 9:30 am or between 12:30 and 1 pm

In [ ]:
gen time_instrument_e=inlist(time_hr,1,8) if !missing(time_hr) 
ivregress 2sls seconcilio treatment2 $controls (1.p_actor 1.p_actor#1.treatment2 = 1.time_instrument_e 1.time_instrument_e#1.treatment2), first

## Instrumented by "extreme" early hours: before 10 am

In [ ]:
gen time_instrument_early=inlist(time_hr,1,2) if !missing(time_hr) 
ivregress 2sls seconcilio treatment2 $controls (1.p_actor 1.p_actor#1.treatment2 = 1.time_instrument_early 1.time_instrument_early#1.treatment2), first

## Instrumented by "extreme" super early hours: before 9:30 am

In [ ]:
gen time_instrument_Searly=inlist(time_hr,1) if !missing(time_hr) 
ivregress 2sls seconcilio treatment2 $controls (1.p_actor 1.p_actor#1.treatment2 = 1.time_instrument_Searly 1.time_instrument_Searly#1.treatment2), first

## Instrumented by "extreme" late hours: between 12:30 and 13

In [ ]:
gen time_instrument_Slate=inlist(time_hr,8) if !missing(time_hr) 
ivregress 2sls seconcilio treatment2 $controls (1.p_actor 1.p_actor#1.treatment2 = 1.time_instrument_Slate 1.time_instrument_Slate#1.treatment2), first

## Instrumented by "extreme" hours: before 9:30 or between 12:30 and 13

In [ ]:
gen time_instrument_extreme=inlist(time_hr,1,8) if !missing(time_hr) 
ivregress 2sls seconcilio treatment2 $controls (1.p_actor 1.p_actor#1.treatment2 = 1.time_instrument_extreme 1.time_instrument_extreme#1.treatment2), first

## Instrumented by earlt hours: before 11 am (the median)

In [ ]:
h ivregress

In [ ]:
gen time_instrument_leqMedian=time_hr<5 if !missing(time_hr) 
ivregress 2sls seconcilio treatment2 $controls (1.p_actor 1.p_actor#1.treatment2 = 1.time_instrument_leqMedian 1.time_instrument_leqMedian#1.treatment2), first
